In [112]:
import helpers
import numpy as np
import re 

#helpers.npNicePrint(arr)

data = """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""

with open("input_15.txt", 'r') as f:
    data = f.read()


def safe_get(grid, index):
    try:
        return grid[tuple(index)]
    except:
        return ""
    
#changeDict = {"forward": np.array([1,0]), "down": np.array([0,1]), "up": np.array([0,-1])}
#data = [changeDict[x.split(" ")[0]] * int(x.split(" ")[1]) for x in data.split("\n") if x]
#data = [
#    [int(match.group(1)), list(map(int, match.group(2).split()))]
#    for line in data.split("\n")
#    if (match := re.match(r'^(\d+):\s*(\d+(?:\s+\d+)*)', line))
#]
#np.prod(sum(data))    
#data = [list(map(int, x.split(" "))) for x in data.split('\n') if not x == ""]

grid, moves = data.split("\n\n")
moves = moves.replace("\n", "")

grid = grid.replace("#", "##")
grid = grid.replace("O", "[]")
grid = grid.replace(".", "..")
grid = grid.replace("@", "@.")

grid = {(x,y): str(val) for y, r in enumerate(grid.split('\n')) for x, val in enumerate(r)}
#grid
#data = [int(x) for x in data.split(" ")]




# Output the parsed data
#print(data)

#helpers.visualize_dict_grid(grid) 

In [113]:
moves_dx = {"<": np.array([-1,0]),">": np.array([1,0]),"^": np.array([0,-1]),"v": np.array([0,1])} 

location = [key for key, value in grid.items() if value == "@"][0]

def ball_move(location, move, n_grid): #sorta double but w/e
    #print(f"ball_move called: {location}, {move}")
    if move == "<":
        #print(tuple(location[0] + moves_dx[move])
        #print(n_grid[tuple([location[0] + moves_dx[move]])])
        new_location = [location[0] + moves_dx[move]]
        contents = [n_grid[tuple(new_location[0])]]
        
    elif move == ">":
        new_location = [location[1] + moves_dx[move]]
        contents = [n_grid[tuple(new_location[0])]]
    else:
        new_location = [loc + moves_dx[move] for loc in location]
        contents = [n_grid[tuple(new_loc)] for new_loc in new_location]
    #print(new_location, contents)
    can_moves = []
    #print(new_location, contents)
    if len(contents) > 1:
        if contents[0] == "[" and contents[1] == "]":#if it's one box then don't do it twice
            contents = [contents[1]]
            new_location = [new_location[1]]
            #print(new_location, contents)
    for content, new_location_2 in zip(contents, new_location):
        #print("looping through move location, creating new boxes if neccesary")
        #print(content, new_location_2)
        if content == "#": #nothing happens
            can_moves.append(False)
        elif content == ".": #simple move
            can_moves.append(True)
        elif content == "[" or content == "]": #ball movement
            if content == "[":
                new_box_location = [new_location_2, new_location_2 + np.array([1,0])] 
            elif content == "]":
                new_box_location = [new_location_2 + np.array([-1,0]), new_location_2]
            moved, n_grid  = ball_move(new_box_location, move, n_grid)   
            if moved:
                can_moves.append(True)
            else:
                can_moves.append(False)
    #print(can_moves)            
    if all(can_moves):
        n_grid[tuple(location[0])] = "."
        n_grid[tuple(location[1])] = "."
        n_grid[tuple(location[0] + moves_dx[move])] = "["
        n_grid[tuple(location[1] + moves_dx[move])] = "]"
        return True, n_grid
    return False, n_grid

for move in moves:
    #print(move)
    new_location = location + moves_dx[move]
    contents = grid[tuple(new_location)]
    n_grid = grid.copy()
    if contents == "#": #nothing happens
        pass 
    elif contents == ".": #simple move
        grid[tuple(location)] = "."
        grid[tuple(new_location)] = "@"
        location = new_location
    elif contents == "[" or contents == "]": #ball movement
        #print("ya", new_location)
        if contents == "[":
            box_location = [new_location, new_location + np.array([1,0])] 
        elif contents == "]":
            box_location = [new_location + np.array([-1,0]), new_location] 
        #print(box_location)
        moved, n_grid = ball_move(box_location, move, grid.copy())
        if moved:
            grid = n_grid
            grid[tuple(location)] = "."
            grid[tuple(new_location)] = "@"
            location = new_location
#helpers.visualize_dict_grid(grid) 

ans = [key for key, value in grid.items() if value == "["]
sum([x + 100* y for x,y in ans])
        

1495455